<a href="https://colab.research.google.com/github/JuliustheCreator/channel-growth-model/blob/main/analysis/youtube_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Importing Modules**


---



In [ ]:
import pandas as pd
import tensorflow as tf
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from google.colab import files

#### **Importing Cleaned Dataset**


---



In [ ]:
uploaded = files.upload()

with open("cleaned_data.pkl", "rb") as f:
    df = pickle.load(f)


Saving cleaned_data.pkl to cleaned_data.pkl


### **Cleaning Dataset Further for Model**

In [ ]:
# Removing channels (rows) where there are no videos or no views
df = df.drop(df[(df['Video Views'] == 0) | (df['Video Count'] == 0)].index)

# Augmenting dataset (adding dummy channels)
for i in range(250):

  # Randomizing age around the mean
  age = np.random.normal(loc = df['Age'].mean(), scale = df['Age'].std())

  dummy_channel = pd.DataFrame({'Youtube Channel': ['Dummy Channel'], 'Subscribers':[0], 'Video Views':[0], 'Video Count':[0], 'Age':[int(age)]})

  df = pd.concat([df, dummy_channel], ignore_index = True)

### **Building the Regression Model (Neural Network)**


---



In [ ]:
# Selecting required columns
X = df[['Video Views', 'Video Count', 'Age']]
y = df['Subscribers']

# Scaling the features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 220)

# Creating Model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation = 'relu', input_shape = [X_train.shape[1]]),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(1)
])

### **Training the Model**


---



In [ ]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

history = model.fit(X_train, y_train, epochs = 100, batch_size = 50, validation_split = 0.2)

Epoch 1/100
16/16 [==============================] - 1s 14ms/step - loss: 516.3254 - val_loss: 824.3991
Epoch 2/100
16/16 [==============================] - 0s 5ms/step - loss: 507.0865 - val_loss: 812.0538
Epoch 3/100
16/16 [==============================] - 0s 5ms/step - loss: 495.1099 - val_loss: 794.7263
Epoch 4/100
16/16 [==============================] - 0s 5ms/step - loss: 477.9351 - val_loss: 769.0967
Epoch 5/100
16/16 [==============================] - 0s 5ms/step - loss: 452.0368 - val_loss: 732.0414
Epoch 6/100
16/16 [==============================] - 0s 4ms/step - loss: 416.2325 - val_loss: 680.8130
Epoch 7/100
16/16 [==============================] - 0s 4ms/step - loss: 370.3120 - val_loss: 619.4609
Epoch 8/100
16/16 [==============================] - 0s 4ms/step - loss: 319.6981 - val_loss: 554.1934
Epoch 9/100
16/16 [==============================] - 0s 4ms/step - loss: 272.5513 - val_loss: 497.3525
Epoch 10/100
16/16 [==============================] - 0s 6ms/step - loss

### **Evaluating the Model**


---



In [ ]:
loss = model.evaluate(X_test, y_test)
print(f"Mean squared error: {loss}")

8/8 [==============================] - 0s 2ms/step - loss: 45.3746
Mean squared error: 45.37457275390625


### **Exporting and Testing Model**


---



In [ ]:
# To Test the Model with Custom Values
views = float(input('Input Video Views: ')) / 1_000_000
videos = float(input('Input Video Count: ')) / 1000
age = float(input('Input Age of Channel: '))

# Prediction
test_values = scaler.transform(np.array([[views, videos, age]]))

predicted_subscribers = model.predict(test_values)
print(f"Estimated Subscriber Count: {int(predicted_subscribers[0][0]) * 1_000_000:,}")

Input Video Views: 1000000
Input Video Count: 500
Input Age of Channel: 5
1/1 [==============================] - 0s 20ms/step
Estimated Subscriber Count: 7,000,000


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
